<a href="https://colab.research.google.com/github/violjii/ML/blob/main/8_2_Serhiienko_FIT_4_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras_tuner as kt

# 1. Функція для побудови моделі для Keras Tuner
def build_model(hp):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))

    # Налаштування кількості нейронів у першому шарі
    model.add(layers.Dense(units=hp.Int('units', min_value=64, max_value=256, step=32), activation='relu'))
    model.add(layers.Dropout(0.3))

    # Налаштування кількості нейронів у другому шарі
    model.add(layers.Dense(units=hp.Int('units2', min_value=32, max_value=128, step=16), activation='relu'))

    model.add(layers.Dense(10, activation='softmax'))

    # Налаштування оптимізатора
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 2. Використання Keras Tuner для підбору гіперпараметрів
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, overwrite=True)

tuner.search(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Виведення результатів підбору
best_model = tuner.get_best_models(1)[0]
best_model.summary()

# 3. Оцінка моделі
best_model.evaluate(x_test, y_test)

# Побудова графіків для нової моделі
history_best = best_model.fit(x_train, y_train, epochs=30, validation_data=(x_test, y_test),
                              callbacks=[early_stop, reduce_lr])

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_best.history['loss'], label='Train Loss')
plt.plot(history_best.history['val_loss'], label='Validation Loss')
plt.title('Функція втрат (підбір)')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_best.history['accuracy'], label='Train Accuracy')
plt.plot(history_best.history['val_accuracy'], label='Validation Accuracy')
plt.title('Точність (підбір)')
plt.legend()
plt.show()
